# Running queries on the OMOP data

## Install required packages

Before executing the cells below, in a terminal session install the following required R packages:

`conda create -n omop-source r-glue r-tidyverse r-data.table r-dbi  r-rpostgres r-irkernel -y`

### Connect to the SQL database

In [ ]:
library(tidyverse)
library(data.table)
library(glue)
library(DBI)
library(RPostgres)

DBNAME <- #
HOST <- #
PORT <- #
PASSWORD <-  #
USER <- 'jupyter_notebook'

connection <- DBI::dbConnect(
    RPostgres::Postgres(),
    dbname = DBNAME,
    host = HOST,
    port = PORT,
    password = PASSWORD,
    user = USER,
    )

# SQL query - Demographics
This is a simple query that provides the total number of participants with a valid VCF path

In [ ]:
omop_query <- "
    select count(distinct o.person_id) as count
    from omop_data_serena_v1.observation o
    left join omop_data_serena_v1.person p on o.person_id = p.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = p.gender_concept_id
    inner join omop_data_serena_v1.observation o2 on o.person_id = o2.person_id and o2.observation_source_value = 'VCF_index_file_path_URI' and o2.value_as_string IS NOT NULL
    where o.observation_concept_id = '4161164'
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo

In [ ]:
This is a simple query that provides the total of the counts stratified by gender

In [ ]:
omop_query <- "
    select count(distinct p.person_id) as count, c.concept_name, c.concept_id
    from omop_data_serena_v1.observation o
    left join omop_data_serena_v1.person p on o.person_id = p.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = p.gender_concept_id
    inner join omop_data_serena_v1.observation o2 on o.person_id = o2.person_id and o2.observation_source_value = 'VCF_index_file_path_URI' and o2.value_as_string IS NOT NULL
    where o.observation_concept_id = '4161164'
    group by c.concept_name, c.concept_id
    order by count desc
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo

This is a simple query that provides the total of the counts stratified by gender and organ

In [ ]:
omop_query <- "
    select count(distinct p.person_id) as count, c.concept_name, c.concept_id, o.value_as_string as ORGAN
    from omop_data_serena_v1.observation o
    left join omop_data_serena_v1.person p on o.person_id = p.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = p.gender_concept_id
    inner join omop_data_serena_v1.observation o2 on o.person_id = o2.person_id and o2.observation_source_value = 'VCF_index_file_path_URI' and o2.value_as_string IS NOT NULL
    where o.observation_concept_id = '4161164'
    group by c.concept_name, c.concept_id, o.value_as_string
    order by count desc
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo

# SQL query part 2
This query pulls all the information needed (ID, ORGAN, VCF) for the mut-sig pipeline.

In [ ]:
omop_query <- "
    select distinct p.person_source_value as ID, o.value_as_string as ORGAN, o2.value_as_string as VCF
    from omop_data_serena_v1.observation o
    left join omop_data_serena_v1.person p on o.person_id = p.person_id
    inner join omop_data_serena_v1.observation o2 on o.person_id = o2.person_id and o2.observation_source_value = 'VCF_file_path_URI' and o2.value_as_string IS NOT NULL
    where o.observation_concept_id = '4161164'
    and o2.value_as_string like '%.snv.simple.txt.vcf.gz'
    limit 10
"

all_dataset <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset

# SQL query part 3
This query filters by organ

This query filters by organ. Where organ equals to breast.

In [ ]:
omop_query <- "
    select p.person_source_value, c.concept_name, c.concept_id, o.value_as_string as ORGAN
    from omop_data_serena_v1.observation o
    left join omop_data_serena_v1.person p on o.person_id = p.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = p.gender_concept_id
    inner join omop_data_serena_v1.observation o2 on o.person_id = o2.person_id and o2.observation_source_value = 'VCF_index_file_path_URI' and o2.value_as_string IS NOT NULL
    where o.observation_concept_id = '4161164' and o.value_as_string = 'Breast'
    limit 10
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo

This query filters the participants by the ICD10 code C61 or Primary malignant neoplasm of prostate. 

In [ ]:
omop_query <- "
    select p.person_source_value,co.condition_concept_id, co.condition_source_value, c.concept_name
    from omop_data_serena_v1.condition_occurrence co
    left join omop_data_serena_v1.person p on p.person_id = co.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = co.condition_concept_id
    where co.condition_source_value = 'C61'
    limit 20
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo

This query filters the participants by the exposure to Nivolumab

In [ ]:
omop_query <- "
    select p.person_source_value,co.drug_concept_id, co.drug_source_value, c.concept_name
    from omop_data_serena_v1.drug_exposure co
    left join omop_data_serena_v1.person p on p.person_id = co.person_id
    left join omop_data_serena_v1.concept c on c.concept_id = co.drug_concept_id
    where drug_concept_id = '45892628'
    limit 20
"

all_dataset_demo <- dbGetQuery(connection, glue(omop_query, schema = "omop_data_serena_v1"))

all_dataset_demo